# Imports

In [ ]:
import packages
from configs import settings, const, components
from configs.settings import logger
import asyncio, os, time, yaml, json, datetime, copy, random
from typing import Any, AsyncGenerator, Generator, Callable, Literal, Optional, TypeAlias, Union
from tqdm import tqdm
from pprint import pprint

from toolkit.llm.llama_index import (
	agents, cores, deploys as dpls, evaluation, messages, models, 
	observability, types, utils as utils_llama_index, workflows as wfs
)
from toolkit.llm.llama_index.data import loading, querying, storing

from features.agents.car.tools import VehicleDB
from features.agents.tools import map

from toolkit.utils import utils, typer as t
from toolkit.utils.llm import measure_performance, main as utils_llm
from toolkit.utils.utils import rp_print

# <span style="color: #f59e0b;">Dev</span>
<span style="color: red;">There is no need to run this unless you are testing the framework.</span>

In [ ]:
	def multiply(a: float, b: float) -> float:
		"""
		Multiply two float numbers and returns the product.
		"""
		return a * b

	def add(a: float, b: float) -> float:
		"""
		Add two float numbers and returns the sum.
		"""
		return a + b

	def useless(a: float, b: float) -> float:
		"""
		Toy useless function.
		"""
		pass

tool_multiply = agents.FunctionTool.from_defaults(fn=multiply)
tool_add = agents.FunctionTool.from_defaults(fn=add)
tools_useless = [
	agents.FunctionTool.from_defaults(fn=useless, name=f"useless_{str(idx)}")
	for idx in range(28)
]

tools = agents.add_tools(
	tool_multiply,
	tool_add,
	[tools_useless],
	# query_plan_tool,
	# other agents
)
tools_map = {tool.metadata.name: tool for tool in tools}

tools_obj_index = storing.ObjectIndex.from_objects(
	tools, index_cls=storing.VectorStoreIndex,
)
tool_retriever = tools_obj_index.as_retriever(similarity_top_k=2)

agent = agents.AgentRunner.from_llm(
	# tools=tools, # use this
	tool_retriever=tool_retriever, # or this
	llm=cores.Settings.llm,
	verbose=True,
)

# agent.achat()

# agent = agents.FunctionCallingAgent.from_tools(
# 	tools=tools,
# 	llm=cores.Settings.llm,
# 	verbose=True,
# 	system_prompt=None,
# 	allow_parallel_tool_calls=True,
# )

# worker = agents.FunctionCallingAgentWorker.from_tools(tools=tools, verbose=True)
# agent = agents.StructuredPlannerAgent(worker, tools=tools, verbose=True) # take longer to respond

# agent = agents.ReActAgent.from_tools(
#   tools=tools, 
# 	llm=cores.Settings.llm,
# 	verbose=True,
# 	context=None, # System prompt
# )

# agent_worker_openai = agents.OpenAIAgentWorker.from_tools(tools=tools, verbose=True)
# agent_openai = agents.AgentRunner(agent_worker_openai)

In [ ]:
queries = [
	"What is 2123 * 215123",
	"What is 20+(2*4)? Use a tool to calculate every step.",
	"What is (121 * 3) + 42?",
	"What is (121 * 3) + (5 * 8)?",
	("What is 121 * 2? Once you have the answer, "
   "use that number to write a story about a group of mice."),
	"What's 212 multiplied by 122? Make sure to use Tools",
	"What's 212 added to 122 ? Make sure to use Tools",	
]

In [ ]:
# Usage example:
result = await utils_llama_index.interact_agent(
    agent=agent,
    user_query=queries[0],
    mode="achat",
)

final_result = await utils_llama_index.handle_agent_response(result)

In [ ]:
# Step-Wise Execution
task = agent.create_task(queries[2])
step_output = agent.run_step(task.task_id)

if not step_output.is_last:
	step_output = agent.run_step(task.task_id)

result = agent.finalize_response(task.task_id)
result = str(result)

print(f"\n{'-'*80}\n")
pprint(result)

# print(f"Tasks: {task.list_tasks()}")
# print(f"Completed steps: {task.get_completed_steps(task.task_id)}")

## My Agent Worker

In [ ]:
class AgentWorkerCustom(agents.CustomSimpleAgentWorker):
	"""
	Agent worker.
	Continues iterating until there's no errors / task is done.
 	"""

	def __init__(
		self,
		tools: t.List[agents.BaseTool],
		**kwargs: Any,
	) -> None:
		"""
		Init params.
   	"""

		self.tools = tools

		# Validate tools

		super().__init__(
			tools=self.tools,
			**kwargs,
		)

	def _initialize_state(
		self, task: agents.Task, **kwargs: t.Any,
	) -> t.Dict[str, t.Any]:
		"""
  	Initialize state.
   	"""
		return {
			"count": 0,
			"current_reasoning": [],
		}
	
	def _run_step(
		self, state: t.Dict[str, t.Any], task: agents.Task, 
  	input: t.Optional[str] = None,
	) -> t.Tuple[agents.AgentChatResponse, bool]:
		"""
		Run step.
   	"""
		new_input = state.get("new_input", task.input)

		response = self._get_response()
		
		# Append to current reasoning
		state["current_reasoning"].extend(
			[
				("user", new_input),
				("assistant", str(response)),
			]
		)
	
	def _get_response(
		self, user_query: str,
	):
		return "Hello"
	
